In [1]:
import pandas as pd
import numpy as np
pd.set_option('max_columns', 50)
from datetime import datetime

In [2]:
df = pd.read_csv('Data Set 2.csv')
df.head()

,OrderId,CustomerId,LocationId,channel,Amount,OrderDate,BillNo
0,35623100,6662,4416,foodpanda,267,1/1/2018,DEL
1,35622814,8237,4002,zomato,1051,1/1/2018,DEL
2,35623284,11912,4416,takeaway,714,1/1/2018,DEL
3,35631171,14563,218,zomato,834,1/1/2018,GGN
4,35635515,14833,295,call,544,1/1/2018,DEL


In [3]:
df['OrderDate']= pd.to_datetime(df['OrderDate'])

In [4]:
df['OrderPeriod'] = df.OrderDate.apply(lambda x: x.strftime('%Y-%m'))

In [5]:
df.head()

,OrderId,CustomerId,LocationId,channel,Amount,OrderDate,BillNo,OrderPeriod
0,35623100,6662,4416,foodpanda,267,2018-01-01,DEL,2018-01
1,35622814,8237,4002,zomato,1051,2018-01-01,DEL,2018-01
2,35623284,11912,4416,takeaway,714,2018-01-01,DEL,2018-01
3,35631171,14563,218,zomato,834,2018-01-01,GGN,2018-01
4,35635515,14833,295,call,544,2018-01-01,DEL,2018-01


In [6]:
#Revenue
RevByMonth = df.groupby('OrderPeriod')['Amount'].sum() 
RevByMonth = pd.DataFrame(RevByMonth)
RevByMonth.head()

,Amount
OrderPeriod,
2018-01,14386922
2018-02,14527317
2018-03,14466406
2018-04,14813891
2018-05,15520988


In [7]:
#Orders
OrderByMonth = df.groupby('OrderPeriod')['OrderId'].count() #Orders
OrderByMonth = pd.DataFrame(OrderByMonth)
OrderByMonth.head()

,OrderId
OrderPeriod,
2018-01,23734
2018-02,23593
2018-03,24865
2018-04,25760
2018-05,27844


In [8]:
#Customers
CusByMonth = df.groupby("OrderPeriod").agg({"CustomerId": pd.Series.nunique}) 
CusByMonth.head()

,CustomerId
OrderPeriod,
2018-01,17181
2018-02,17078
2018-03,17212
2018-04,18114
2018-05,19786


In [9]:
#Repeat Customers
grouped = df.groupby(['OrderPeriod', 'CustomerId'])
RC = grouped.agg({'OrderId': pd.Series.nunique})
RC.head()

OrderId
OrderPeriod CustomerId         
2018-01     134               1
            180               1
            1001              1
            1322              1
            1541              1

In [10]:
RCNew = RC[RC.OrderId > 1]
RCNew.head()

OrderId
OrderPeriod CustomerId         
2018-01     4593              4
            5838              2
            6172              2
            6423              2
            6751              2

In [11]:
RepCustomers = RCNew.groupby('OrderPeriod').count() 
RepCustomers.rename(columns={'OrderId': 'RepeatCustomers'}, inplace=True)
RepCustomers.head()

,RepeatCustomers
OrderPeriod,
2018-01,3657
2018-02,3789
2018-03,4029
2018-04,4181
2018-05,4607


In [12]:
#New Customers
df.set_index('CustomerId', inplace=True)
df['CohortGroup'] = df.groupby(level=0)['OrderDate'].min().apply(lambda x: x.strftime('%Y-%m'))
df.reset_index(inplace=True)
df.head()

,CustomerId,OrderId,LocationId,channel,Amount,OrderDate,BillNo,OrderPeriod,CohortGroup
0,6662,35623100,4416,foodpanda,267,2018-01-01,DEL,2018-01,2018-01
1,8237,35622814,4002,zomato,1051,2018-01-01,DEL,2018-01,2018-01
2,11912,35623284,4416,takeaway,714,2018-01-01,DEL,2018-01,2018-01
3,14563,35631171,218,zomato,834,2018-01-01,GGN,2018-01,2018-01
4,14833,35635515,295,call,544,2018-01-01,DEL,2018-01,2018-01


In [13]:
grouped = df.groupby(['CohortGroup', 'OrderPeriod'])
newC = grouped.agg({'CustomerId': pd.Series.nunique,
                       'OrderId': pd.Series.nunique})
newC.head()

CustomerId  OrderId
CohortGroup OrderPeriod                     
2018-01     2018-01           17181    23734
            2018-02            4041     7505
            2018-03            3540     6604
            2018-04            3146     5784
            2018-05            2799     5085

In [14]:
def cohort_period(df):
    
    df['CohortPeriod'] = np.arange(len(df)) + 1
    return df

newC = newC.groupby(level=0).apply(cohort_period)
newC.head()

CustomerId  OrderId  CohortPeriod
CohortGroup OrderPeriod                                   
2018-01     2018-01           17181    23734             1
            2018-02            4041     7505             2
            2018-03            3540     6604             3
            2018-04            3146     5784             4
            2018-05            2799     5085             5

In [15]:
NewCustomers = newC[newC['CohortPeriod'] == 1]

In [16]:
NC = NewCustomers.reset_index()

In [17]:
NC = NC.drop(['CohortGroup'], axis =1)

In [18]:
NC = NC.set_index('OrderPeriod')

In [19]:
NC = NC[['CustomerId']]

In [20]:
NC.rename(columns={'CustomerId': 'NewCustomers'}, inplace=True)

In [21]:
NC.head()

,NewCustomers
OrderPeriod,
2018-01,17181
2018-02,13037
2018-03,11657
2018-04,11473
2018-05,12790


In [22]:
dfMerge = pd.concat([RevByMonth, OrderByMonth, CusByMonth, NC,RepCustomers],axis=1,sort=False).reset_index()
dfMerge.rename(columns = {'index':'OrderPeriod'})
dfMerge.rename(columns={'Amount': 'Revenue',
                        'OrderId': 'Orders',
                        'CustomerId':'Customers'}, inplace=True)

In [23]:
dfMerge['Frequency'] = dfMerge['Orders']/dfMerge['Customers']

In [24]:
dfMerge.head()

,OrderPeriod,Revenue,Orders,Customers,NewCustomers,RepeatCustomers,Frequency
0,2018-01,14386922,23734,17181,17181,3657,1.381410
1,2018-02,14527317,23593,17078,13037,3789,1.381485
2,2018-03,14466406,24865,17212,11657,4029,1.444632
3,2018-04,14813891,25760,18114,11473,4181,1.422104
4,2018-05,15520988,27844,19786,12790,4607,1.407258


In [25]:
dfMerge

,OrderPeriod,Revenue,Orders,Customers,NewCustomers,RepeatCustomers,Frequency
0,2018-01,14386922,23734,17181,17181,3657,1.381410
1,2018-02,14527317,23593,17078,13037,3789,1.381485
2,2018-03,14466406,24865,17212,11657,4029,1.444632
3,2018-04,14813891,25760,18114,11473,4181,1.422104
4,2018-05,15520988,27844,19786,12790,4607,1.407258
5,2018-06,16853200,29488,21234,14105,5004,1.388716
6,2018-07,19219256,32578,23862,15967,5486,1.365267
7,2018-08,15657494,26533,20230,12343,3983,1.311567
8,2018-09,16686127,35130,25731,17193,5633,1.365279
9,2018-10,23063965,39498,28381,19395,6328,1.391706


In [26]:
dfMerge = dfMerge.set_index('OrderPeriod')

In [27]:
dfMerge.transpose()

OrderPeriod,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04
Revenue,1.438692e+07,1.452732e+07,1.446641e+07,1.481389e+07,1.552099e+07,1.685320e+07,1.921926e+07,1.565749e+07,1.668613e+07,2.306396e+07,3.171771e+07,2.590657e+07,1.823194e+07,1.954216e+07,2.639018e+07,7.092316e+06
Orders,2.373400e+04,2.359300e+04,2.486500e+04,2.576000e+04,2.784400e+04,2.948800e+04,3.257800e+04,2.653300e+04,3.513000e+04,3.949800e+04,5.203600e+04,4.201700e+04,3.057400e+04,4.087400e+04,6.091200e+04,1.789100e+04
Customers,1.718100e+04,1.707800e+04,1.721200e+04,1.811400e+04,1.978600e+04,2.123400e+04,2.386200e+04,2.023000e+04,2.573100e+04,2.838100e+04,3.695200e+04,3.123900e+04,2.296500e+04,3.092200e+04,4.413100e+04,1.476200e+04
NewCustomers,1.718100e+04,1.303700e+04,1.165700e+04,1.147300e+04,1.279000e+04,1.410500e+04,1.596700e+04,1.234300e+04,1.719300e+04,1.939500e+04,2.460500e+04,1.860300e+04,1.237000e+04,1.764100e+04,2.632900e+04,7.309000e+03
RepeatCustomers,3.657000e+03,3.789000e+03,4.029000e+03,4.181000e+03,4.607000e+03,5.004000e+03,5.486000e+03,3.983000e+03,5.633000e+03,6.328000e+03,8.844000e+03,6.549000e+03,4.569000e+03,6.209000e+03,9.865000e+03,2.261000e+03
Frequency,1.381410e+00,1.381485e+00,1.444632e+00,1.422104e+00,1.407258e+00,1.388716e+00,1.365267e+00,1.311567e+00,1.365279e+00,1.391706e+00,1.408205e+00,1.345017e+00,1.331330e+00,1.321842e+00,1.380254e+00,1.211963e+00


In [28]:
dfMerge.to_excel(r'C:\Users\user\Desktop\Xeno\MIS Report.xlsx')